In [ ]:
"""Nikita Istomin - CS4375 - Assignment 3 - Tweets Clustering using k-means
    Date: April 23rd, 2023
"""

In [ ]:
import numpy as np
import pandas as pd
import random
import time
import math
import matplotlib.pyplot as plt
from urllib.request import urlopen

plt.style.use(['ggplot'])
%matplotlib inline

In [ ]:

def fetch_dataset(url: str) -> None:
    """Fetch a txt dataset from a public url and return it as a numpy array.

    Args:
        url (str): public url to a txt dataset

    Returns:
        dataset (np.array): dataset as a numpy array
    """
    with urlopen(url=url) as data:
        dataset = np.array(data.read().decode('utf-8').splitlines())
    return dataset

In [ ]:
def preprocess(raw_data: np.ndarray) -> None:
        """Preprocess the dataset

        Args:
            raw_data (np.ndarray): raw dataset as a numpy array

        Returns:
            data (np.ndarray): preprocessed dataset as a numpy array
        """
        """
        Perform the following pre-processing steps:
            1) Remove the tweet id and timestamp
            2) Remove any word that starts with the symbol @ e.g. @AnnaMedaris
            3) Remove any hashtag symbols e.g. convert #depression to depression
            4) Remove any URL
            5) Convert every word to lowercase
        """
        data = []
        for tweet in raw_data:
            tweet = tweet.split('|')
            if len(tweet) != 3:
                continue # skip tweets that don't have 3 parts
            tweet = tweet[2] # remove tweet id and timestamp
            tweet = tweet.split() # split tweet into words
            tweet = [word.lower() for word in tweet if not word.startswith('@')] # remove words that start with @
            tweet = [word[1:] if word.startswith('#') else word for word in tweet] # remove hashtag symbol
            tweet = [word for word in tweet if not word.startswith('http')] # remove urls
            # convert tweet to numpy array
            tweet = np.array(tweet)
            data.append(tweet)
        
        data = np.array(data, dtype=object)
        return data

In [ ]:
class KMeans():
    """KMeans clustering algorithm

    Attributes:
        k (int): number of clusters
        max_iter (int): maximum number of iterations
        centroids (np.ndarray): centroid tweets of each cluster
        clusters (np.ndarray): indices of tweets in each cluster
        sse (np.ndarray): history of sum of squared errors (SSE)
    """
    def __init__(self, k: int, max_iter: int = 10) -> None:
        """Initialize KMeans clustering algorithm

        Args:
            k (int): number of clusters
            max_iter (int): maximum number of iterations
        """
        self.k = k
        self.max_iter = max_iter
        self.centroids = None
        self.clusters = None
        self.sse = np.array([])

    def fit(self, data: np.ndarray) -> None:
        """Fit the data to the KMeans clustering algorithm

        Args:
            data (np.ndarray): dataset as a numpy array
        """
        start = time.time()
        self._initialize_centroids(data)
        finish = time.time()
        print(f'Initialization time: {finish - start} seconds')
        previous_centroids = np.copy(self.centroids)

        for i in range(self.max_iter):
            print(f'Iteration {i + 1}')

            start = time.time()
            self._assign_clusters(data)
            finish = time.time()
            print(f'Assignment time: {finish - start} seconds')

            start = time.time()
            self._update_centroids(data)
            finish = time.time()
            print(f'Update time: {finish - start} seconds')

            self.sse = np.append(self.sse, self._SSE(data))
            print(f'SSE: {self.sse[-1]}')
            
            # if the centroids don't change, stop the algorithm
            if np.array_equal(self.centroids, previous_centroids):
                break
            previous_centroids = np.copy(self.centroids)

    def visualize(self) -> None:
        """Visualize the SSE history"""
        plt.plot(self.sse)
        plt.xlabel('Iteration')
        plt.ylabel('SSE')
        plt.title('SSE History')
        plt.show()


    def _SSE(self, data: np.ndarray) -> float:
        """Compute the sum of squared errors (SSE)

        Args:
            data (np.ndarray): dataset as a numpy array

        Returns:
            SSE (float): sum of squared errors
        """
        SSE = 0
        for i, cluster in enumerate(self.clusters):
            SSE += np.sum(np.array([self._compute_distance(data[tweet], self.centroids[i])**2 for tweet in cluster]))
        return SSE
        

    def _initialize_centroids(self, data: np.ndarray) -> None:
        """Initialize the centroids

        Args:
            data (np.ndarray): dataset as a numpy array
        """
        # randomly select k data points as initial centroids
        self.centroids = data[np.random.choice(data.shape[0], self.k, replace=False)]

    def _assign_clusters(self, data: np.ndarray) -> None:
        """Assign the data to the closest cluster

        Args:
            data (np.ndarray): dataset as a numpy array
        """
        self.clusters = np.zeros(self.k, dtype=np.ndarray)
        for i, tweet in enumerate(data):
            # compute the distance between the tweet and each centroid
            distances = np.zeros(self.k)
            for j, centroid in enumerate(self.centroids):
                start = time.time()
                distances[j] = self._compute_distance(tweet, centroid)
                finish = time.time()
                # print(f'Distance {j+1} computation time: {finish - start} seconds')
            # assign the tweet to the closest cluster
            start = time.time()
            min_distance = np.argmin(distances)
            self.clusters[min_distance] = np.append(self.clusters[min_distance], i)
            finish = time.time()
            # print(f'Cluster assignment time: {finish - start} seconds')


    def _update_centroids(self, data: np.ndarray) -> None:
        """Update the centroids to the tweet with the lowest distance to other tweets in the cluster

        Args:
            data (np.ndarray): dataset as a numpy array
        """
        for i, cluster in enumerate(self.clusters):
            # compute the distance between each tweet in the cluster and every other tweet in the cluster
            start = time.time()
            distances = np.array([[self._compute_distance(data[tweet1], data[tweet2]) for tweet2 in cluster] for tweet1 in cluster])
            finish = time.time()
            print(f'Distance {i+1} computation time: {finish - start} seconds')
            # update the centroid to the tweet with the lowest mean of distances to other tweets in the cluster
            self.centroids[i] = data[cluster[np.argmin(np.mean(distances, axis=1))]]
        
    def _compute_distance(self, x: np.ndarray, y: np.ndarray) -> float:
        """Compute the Jaccard Distance between two points

        Args:
            x (np.ndarray): first point
            y (np.ndarray): second point

        Returns:
            distance (float): Jaccard Distance between two points
        """
        # Jacard Distance = 1 - (intersection of two sets / union of two sets)

        distance = 1 - (np.intersect1d(x, y).size / np.union1d(x, y).size)
        return distance



In [ ]:
class KMeansGrid():
    def __init__(self, k: list, max_iter: list) -> None:
        self.params = {'k': k, 'max_iter': max_iter}
        self.models = []
        self.table = pd.DataFrame(columns=['k', 'SSE', 'Cluster Size'])
    def run(self, data: np.ndarray) -> None:
        for k in self.params['k']:
            for max_iter in self.params['max_iter']:
                print(f'k = {k}, max_iter = {max_iter}')
                kmean = KMeans(k=k, max_iter=max_iter)
                self.models.append(kmean)
                kmean.fit(data)
                kmean.visualize()
                self.table = self.table.append(pd.DataFrame({'k': kmean.k, 'SSE': kmean.sse[-1], 'Cluster Size': [[cluster.size for cluster in kmean.clusters]]}))

In [ ]:
# Initialize KMeans clustering algorithm
params = {'k': [3, 5, 7, 10, 15], 'max_iter': [5]}
kmeans = KMeansGrid(**params)
# Fetch dataset
print("Fetching dataset... ", end="")
dataset: np.ndarray = fetch_dataset(url='https://raw.githubusercontent.com/NorthPhoenix/ML-Tweets-Clustering-using-k-means/main/Datasets/usnewshealth.txt')

print("Done")

In [ ]:
# Preprocess dataset
print("Preprocessing dataset... ", end="")
dataset = preprocess(raw_data=dataset)
print("Done")

In [ ]:
print("Fitting dataset... ", end="")
kmeans.run(dataset)
print("Done")

In [400]:
print(kmeans.table)

    k          SSE                                       Cluster Size
0   3  1168.822648                                    [839, 418, 146]
0   5  1140.615147                          [255, 521, 186, 156, 287]
0   7  1137.947644                   [400, 472, 42, 199, 74, 99, 121]
0  10  1128.161758       [239, 301, 244, 35, 91, 282, 20, 54, 78, 66]
0  15  1100.883599  [145, 97, 82, 49, 144, 126, 300, 39, 18, 83, 1...
